In [77]:
import sys
sys.path.append('../../')
from brokers.coinbase import Coinbase
import datetime

cb = Coinbase()
symbol = 'BTC/USD'
since = cb.exchange.milliseconds() - 31540000000

# print(cb.exchange.fetch_my_trades(symbol))
# print(cb.exchange.fetch_balance())
# print(cb.exchange.create_market_buy_order(symbol, '0.001'))
# print(cb.exchange.fetch_my_trades(symbol))
# print(cb.exchange.fetch_open_orders(symbol, limit = 5))
# print(cb.exchange.fetch_closed_orders(symbol, limit=5))
print(cb.exchange.fetchOHLCV(symbol, since=since, timeframe = '5m'))

[[1577861400000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577861700000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577862000000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577862300000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577862600000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577862900000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577863200000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577863500000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577863800000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577864100000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577864400000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577864700000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577865000000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577865300000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577865600000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577865900000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577866200000, 7535.99, 7535.99, 101.0, 7535.0, 0.01], [1577866500000, 7535.99, 7535.99, 101.0, 101.0, 0.01], [1577866